In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns
import numpy as np


In [2]:
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import cross_validate

regr = LinearRegression()
treereg = tree.DecisionTreeRegressor()

In [3]:
dfrent = pd.read_csv('../Project-5-scikitlearn/rent.csv',index_col=False)
dfsale = pd.read_csv('../Project-5-scikitlearn/sale.csv',index_col=False)

dfsale.drop(columns = ['Unnamed: 0','propertyCode','operation'],inplace =True)
dfrent.drop(columns = ['Unnamed: 0','propertyCode','operation'],inplace =True)

In [4]:
rentdummy = pd.get_dummies(dfrent)
saledummy = pd.get_dummies(dfsale)

In [5]:
saledummy.head()

,price,size,rooms,bathrooms,latitude,longitude,priceByArea,floorNumeric,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio
0,750000.0,165.0,3,3,38.748643,-9.151992,4545.0,1.0,0,1,0,0
1,235000.0,78.0,2,1,38.749132,-9.104524,3013.0,5.0,0,1,0,0
2,209000.0,38.0,1,1,38.711398,-9.128863,5500.0,3.0,0,1,0,0
3,550000.0,128.0,3,2,38.728655,-9.140998,4297.0,2.0,0,1,0,0
4,455000.0,81.0,2,2,38.741566,-9.144532,5617.0,4.0,0,1,0,0


In [6]:
'''fig, ax = plt.subplots(2,6,sharex='none',figsize=(25,7))
sns.histplot(ax=ax[0,0], data=dfrent['price'],kde=True)
sns.histplot(ax=ax[0,1], data=dfrent['size'],kde=True)
sns.histplot(ax=ax[0,2], data=dfrent['rooms'],kde=True)
sns.histplot(ax=ax[0,3], data=dfrent['priceByArea'],kde=True)
sns.histplot(ax=ax[0,4], data=dfrent['bathrooms'],kde=True)
sns.histplot(ax=ax[0,5], data=dfrent['floorNumeric'],kde=True)


sns.boxplot(ax=ax[1,0],data=dfrent['price'],orient='h')
sns.boxplot(ax=ax[1,1],data=dfrent['size'],orient='h')
sns.boxplot(ax=ax[1,2],data= dfrent['rooms'],orient='h')
sns.boxplot(ax=ax[1,3],data=dfrent['priceByArea'],orient='h')
sns.boxplot(ax=ax[1,4], data=dfrent['bathrooms'],orient='h')
sns.boxplot(ax=ax[1,5], data=dfrent['floorNumeric'],orient='h')'''


"fig, ax = plt.subplots(2,6,sharex='none',figsize=(25,7))\nsns.histplot(ax=ax[0,0], data=dfrent['price'],kde=True)\nsns.histplot(ax=ax[0,1], data=dfrent['size'],kde=True)\nsns.histplot(ax=ax[0,2], data=dfrent['rooms'],kde=True)\nsns.histplot(ax=ax[0,3], data=dfrent['priceByArea'],kde=True)\nsns.histplot(ax=ax[0,4], data=dfrent['bathrooms'],kde=True)\nsns.histplot(ax=ax[0,5], data=dfrent['floorNumeric'],kde=True)\n\n\nsns.boxplot(ax=ax[1,0],data=dfrent['price'],orient='h')\nsns.boxplot(ax=ax[1,1],data=dfrent['size'],orient='h')\nsns.boxplot(ax=ax[1,2],data= dfrent['rooms'],orient='h')\nsns.boxplot(ax=ax[1,3],data=dfrent['priceByArea'],orient='h')\nsns.boxplot(ax=ax[1,4], data=dfrent['bathrooms'],orient='h')\nsns.boxplot(ax=ax[1,5], data=dfrent['floorNumeric'],orient='h')"

In [7]:
x_train, x_test, y_train, y_test = train_test_split(rentdummy[['size', 'rooms', 'bathrooms', 'latitude', 'longitude', 'priceByArea', 'floorNumeric', 'propertyType_duplex','propertyType_flat', 'propertyType_penthouse', 'propertyType_studio']], rentdummy['price'], test_size=0.75,train_size=0.25)

#normalized
x_train_normalized = normalize(x_train)
x_test_normalized = normalize(x_test)

y_train_normalized = normalize(np.array(y_train).reshape(-1,1))
y_test_normalized = normalize(np.array(y_test).reshape(-1,1))



#standardized
x_train_standar = StandardScaler().fit_transform(x_train)
x_test_standar = StandardScaler().fit_transform(x_test)

y_train_standar = StandardScaler().fit_transform(np.array(y_train).reshape(-1,1))
y_test_standar = StandardScaler().fit_transform(np.array(y_test).reshape(-1,1))


xdic = {#'normalized': {'train':x_train_normalized,'test': x_test_normalized},
  
        'standard':{'train':x_train_standar,  'test':x_test_standar},

        'raw':{'train':x_train,'test':x_test} }


ydic = {#'normalized':{'train':y_train_normalized,'test':y_test_normalized},
        
        'standard':{'train':y_train_standar,'test':y_test_standar},
        
        'raw':{'train':y_train,'test':y_test} }

best_acc = []
acc_list = []
for keys in xdic.keys():
    model = regr.fit(xdic[keys]['train'], ydic[keys]['train'])
    cv_acc = cross_validate(model, xdic[keys]['train'], ydic[keys]['train'], cv=10)['test_score'].mean()
    best_acc.append([cv_acc,'regression', keys])
    acc_list.append(cv_acc)
    print(keys, cv_acc)

print('\n')


for keys in xdic.keys():
        for depth in range(5,11):
                treereg = tree.DecisionTreeRegressor(max_depth=depth)
                model = treereg.fit(xdic[keys]['train'],ydic[keys]['train'])
                cv_acc = cross_validate(model, xdic[keys]['train'], ydic[keys]['train'], cv=10)['test_score'].mean()
                best_acc.append([cv_acc,'tree', keys, depth])
                acc_list.append(cv_acc)
                print('depth:',depth,' ',keys, cv_acc)
        print('\n')

for best in best_acc:
        if best[0] == max(acc_list):
                print(best)

standard 0.901551807537588
raw 0.9021640311946652


depth: 5   standard 0.84982176284082
depth: 6   standard 0.8446698168496024
depth: 7   standard 0.8856221986006038
depth: 8   standard 0.8870986428170774
depth: 9   standard 0.8766181012561189
depth: 10   standard 0.8863111207153569


depth: 5   raw 0.8281987573512817
depth: 6   raw 0.8727109527463524
depth: 7   raw 0.8720811744448997
depth: 8   raw 0.8650084678635303
depth: 9   raw 0.8717077981109641
depth: 10   raw 0.8514751275087026


[0.9021640311946652, 'regression', 'raw']


In [12]:
test_rent = pd.read_csv('../Project-5-scikitlearn/rent_test_cleaned.csv')
test_rent.drop(columns = ['Unnamed: 0','propertyCode','operation'],inplace =True)

test_rent_dummy = pd.get_dummies(test_rent)
test_rent_dummy

,price,size,rooms,bathrooms,latitude,longitude,priceByArea,floorNumeric,propertyType_duplex,propertyType_flat,propertyType_penthouse,propertyType_studio
0,1790.0,160.0,4,2,38.711039,-9.130707,11.0,4,1,0,0,0
1,1100.0,56.0,1,1,38.713890,-9.141274,20.0,4,0,1,0,0
2,1950.0,110.0,2,2,38.721599,-9.153305,18.0,2,0,1,0,0
3,1500.0,155.0,3,3,38.747178,-9.167657,10.0,11,0,1,0,0
4,1490.0,100.0,3,2,38.717097,-9.139251,15.0,5,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
104,995.0,55.0,1,2,38.709199,-9.160593,18.0,2,0,1,0,0
105,990.0,90.0,2,1,38.715841,-9.144891,11.0,2,1,0,0,0
106,3000.0,230.0,4,3,38.724447,-9.159055,13.0,1,0,1,0,0
107,3000.0,110.0,1,1,38.713420,-9.145646,27.0,4,0,1,0,0


In [ ]:
test_rent_dummy

In [8]:
x_train, x_test, y_train, y_test = train_test_split(saledummy[['size', 'rooms', 'bathrooms', 'latitude', 'longitude', 'priceByArea', 'floorNumeric', 'propertyType_duplex','propertyType_flat', 'propertyType_penthouse', 'propertyType_studio']], saledummy['price'], test_size=0.75,train_size=0.25)

#normalized
x_train_normalized = normalize(x_train)
x_test_normalized = normalize(x_test)

y_train_normalized = normalize(np.array(y_train).reshape(-1,1))
y_test_normalized = normalize(np.array(y_test).reshape(-1,1))



#standardized
x_train_standar = StandardScaler().fit_transform(x_train)
x_test_standar = StandardScaler().fit_transform(x_test)

y_train_standar = StandardScaler().fit_transform(np.array(y_train).reshape(-1,1))
y_test_standar = StandardScaler().fit_transform(np.array(y_test).reshape(-1,1))


xdic = {#'normalized': {'train':x_train_normalized,'test': x_test_normalized},
  
        'standard':{'train':x_train_standar,  'test':x_test_standar},

        'raw':{'train':x_train,'test':x_test} }


ydic = {#'normalized':{'train':y_train_normalized,'test':y_test_normalized},
        
        'standard':{'train':y_train_standar,'test':y_test_standar},
        
        'raw':{'train':y_train,'test':y_test} }

best_acc = []
acc_list = []
for keys in xdic.keys():
    model = regr.fit(xdic[keys]['train'], ydic[keys]['train'])
    cv_acc = cross_validate(model, xdic[keys]['train'], ydic[keys]['train'], cv=10)['test_score'].mean()
    best_acc.append([cv_acc,'regression', keys])
    acc_list.append(cv_acc)
    print(keys, cv_acc)

print('\n')


for keys in xdic.keys():
        for depth in range(5,11):
                treereg = tree.DecisionTreeRegressor(max_depth=depth)
                model = treereg.fit(xdic[keys]['train'],ydic[keys]['train'])
                cv_acc = cross_validate(model, xdic[keys]['train'], ydic[keys]['train'], cv=10)['test_score'].mean()
                best_acc.append([cv_acc,'tree', keys, depth])
                acc_list.append(cv_acc)
                print('depth:',depth,' ',keys, cv_acc)
        print('\n')

for best in best_acc:
        if best[0] == max(acc_list):
                print(best)

standard 0.9045004620378047
raw 0.9045748845337451


depth: 5   standard 0.8092930475739291
depth: 6   standard 0.8832658478490389
depth: 7   standard 0.8262811728561378
depth: 8   standard 0.8518876184671154
depth: 9   standard 0.8680547707765278
depth: 10   standard 0.8913813712691289


depth: 5   raw 0.8786442014866893
depth: 6   raw 0.8726042014992439
depth: 7   raw 0.8858016052490258
depth: 8   raw 0.8527134300874779
depth: 9   raw 0.9138966413758052
depth: 10   raw 0.8862246900565045


[0.9138966413758052, 'tree', 'raw', 9]


In [9]:
'''fig, ax = plt.subplots(2,4,sharex='none',figsize=(15,7))
sns.histplot(ax=ax[0,0], data=dfsale['price'],kde=True)
sns.histplot(ax=ax[0,1], data=dfsale['size'],kde=True)
sns.histplot(ax=ax[0,2], data=dfsale['rooms'],kde=True)
sns.histplot(ax=ax[0,3], data=dfsale['priceByArea'],kde=True)

sns.boxplot(ax=ax[1,0],data=dfsale['price'],orient='h')
sns.boxplot(ax=ax[1,1],data=dfsale['size'],orient='h')
sns.boxplot(ax=ax[1,2],data= dfsale['rooms'],orient='h')
sns.boxplot(ax=ax[1,3],data=dfsale['priceByArea'],orient='h')'''

"fig, ax = plt.subplots(2,4,sharex='none',figsize=(15,7))\nsns.histplot(ax=ax[0,0], data=dfsale['price'],kde=True)\nsns.histplot(ax=ax[0,1], data=dfsale['size'],kde=True)\nsns.histplot(ax=ax[0,2], data=dfsale['rooms'],kde=True)\nsns.histplot(ax=ax[0,3], data=dfsale['priceByArea'],kde=True)\n\nsns.boxplot(ax=ax[1,0],data=dfsale['price'],orient='h')\nsns.boxplot(ax=ax[1,1],data=dfsale['size'],orient='h')\nsns.boxplot(ax=ax[1,2],data= dfsale['rooms'],orient='h')\nsns.boxplot(ax=ax[1,3],data=dfsale['priceByArea'],orient='h')"